In [1]:
%matplotlib inline
%load_ext Cython
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [2]:
#%%cython -a

from math import sqrt
#from libc.math cimport sqrt
import time
#cimport cython

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class vector:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
    def __mul__(self, other):
        if type(other) == type(self):
            return self.dot(other)
        else:
            return vector(self.x * other, self.y * other, self.z * other)
    def __rmul__(self, other):
        return self.__mul__(other)
    def dot(self, other):
        return vector(self.x * other.x, self.y * other.y, self.z * other.z)
    def __sub__(self, other):
        return vector(self.x - other.x,self.y - other.y,self.z - other.z)
    def __neg__(self):
        return vector(-self.x,-self.y,-self.z)
    def __add__(self, other):
        return vector(self.x + other.x,self.y + other.y,self.z + other.z)
    def mag(self):
        n = self.x*self.x
        n += self.y*self.y
        n += self.z*self.z
        return sqrt(n)
    def mag2(self):
        n = self.x*self.x
        n += self.y*self.y
        n += self.z*self.z
        return abs(n)
    def norm(self):
        return vector(self.x/self.mag(),self.y/self.mag(),self.z/self.mag())            
    def __truediv__(self, other):
        if type(other) == type(self):
            pass
        else:
            return vector(self.x/other, self.y/other, self.z/other)
    def __repr__(self):
        return f'[{self.x},\t{self.y},\t{self.z}]'
    def __print__(self):
        return f'[{self.x},\t{self.y},\t{self.z}]'
    
class planet:
    def __init__(self, name="", pos=[0.0, 0.0, 0.0],velocity=[0.0, 0.0, 0.0], mass=0.0):
        self.name = name
        self.pos = vector(pos[0],pos[1],pos[2])
        self.mass = mass
        self.velocity = vector(velocity[0],velocity[1],velocity[2])
        self.trail_x = [pos[0]]
        self.trail_y = [pos[0]]
        self.trail_z = [pos[0]]
        self.force = vector(0.0,0.0,0.0)
    def update(self):
        self.trail_x.append(self.pos.x)
        self.trail_y.append(self.pos.y)
        self.trail_z.append(self.pos.z)
    def rhat(self):
        return -self.pos.norm()
    def __repr__(self):
        return f'\npos => {self.pos.x},{self.pos.y},{self.pos.z}'
    def __print__(self):
        return f'\npos => {self.pos.x},{self.pos.y},{self.pos.z}'
    
class simulation:
    def __init__(self, sun, planets=[], num=50000, dt=1.0):
        self.sun = sun
        self.planets = planets
        self.num = num
        self.dt = dt
        self.G = 6.67E-11
    def add_planet(self, planet):
        self.planets.append(planet)
    def run(self):
        for p in self.planets:
            for x in range(0, self.num):
                p.force = p.rhat() * (self.G * p.mass * self.sun.mass) / (p.pos.mag2())
                p.velocity = (p.mass * p.velocity + p.force * self.dt) / p.mass
                p.pos = p.pos + (p.velocity * self.dt)
                p.update()
    def plot(self):
        plt.figure(figsize=(15,15))
        axes = plt.gca()
        axes.set_xlabel('x in m')
        axes.set_ylabel('y in m')
        axes.plot(0,0,marker='o',markersize=1,color='red',label='Sun')
        for p in self.planets:
            axes.plot(p.trail_x, p.trail_y,label=p.name)
        plt.axis('equal')
        axes.legend()
        plt.show()
    def plot_3d(self):
        #fig = plt.figure(figsize=(15,15))
        #ax = fig.gca(projection='3d')
        #ax.plot(earth.trail_x, earth.trail_y, earth.trail_z, label='earth')
        #ax.plot(jupiter.trail_x, jupiter.trail_y, jupiter.trail_z, label='jupiter')
        #ax.scatter(sun.trail_x, sun.trail_y, sun.trail_z, label='Sun')
        #plt.axis('equal')
        #ax.legend()
        #plt.show()
        pass

In [5]:
sun = planet(name="sun", mass=2.0E30)
earth = planet(name="earth", pos=[1.5E8,0, 0], velocity=[0, 5.0E5, 0], mass=6.0E24)
jupiter = planet(name="jupiter", pos=[8.0E8,0, 0], velocity=[0, 3.0E5, 0], mass=2.0E27)


sim = simulation(sun, planets=[earth, jupiter], num=1000000)

In [6]:
%%time
sim.run()
#sim.plot()

CPU times: user 29.8 s, sys: 213 ms, total: 30.1 s
Wall time: 30.6 s
